# Structure Property visualizer

This is the second of two deliverables for the SiSc-Lab2020 project.

Authors = 

Supervisors: Dr. Jens Bröder, Dr. Daniel Wortmann, Johannes Wasmer, Prof. Dr. Stefan Blügel.

# Instructions by supervisors

## Jens

You have to implement this notebook

In the end only Markup text and output results should be seen if one hides the code cells (hide_code extension)

That is can easily exported into a nice pdf file.

Also the amount of python code in this notebook should be minimal.

Rather export the functions you use from a different file.

optional dump query results in a file, from which results will be reread for speed, i.e cache results

______________

An example of what we in the end aim for your find here.

https://www.materialscloud.org/discover/mofs#mcloudHeader

Clicking at one of the prerendered plots will open an interactive bokeh app.

Code for this you find here: https://github.com/materialscloud-org/structure-property-visualizer/blob/master/figure/main.py

For you sisc project this is to much, so do not view this as something you have to deliver but something that with the help of your work
in this project we will build out of it for our purposes.

So do not worry about an app.

_________________

**Your tasks are as follows:**

1. Implement a general interactive bokeh scatter plots with linked histograms (see static version examples/MP_convergence_scf_clean_150_240.png). 

   We gladly help you with it. (this goes into helpers.py and you import it here for usage). This function should not contain any aiida methods.
2. Extract float data you find in certain 'Dict' nodes into a/several pandas object, which will be then the datasource for this notebook. 

   So this notebook should not directly depend on any aiida methods (beyond load_profile)


# Imports

In [ ]:
# magics:
# # autoreload imports. 
# # intent: if i change sth in import, i don't have to restart kernel. enable only for development.
%load_ext autoreload
%autoreload 2
# # choose matplotlib backend. backend 'notebook' allows interactive plots if your env allows it.
%matplotlib notebook

In [ ]:
# python imports:
from collections import Counter
import time
#from pprint import pprint

In [ ]:
# aiida imports:
from aiida import load_profile
profile = load_profile()

# ggf add further imports

In [ ]:
# project imports:
import helpers
# from .helpers import bokeh_struc_prop_vis
# from .helpers import generate_structure_property_pandas_source

# equivalent ('.' is the sisc_lab directory):
# from . import helpers
# alternative:
# from .helpers import print_bold
# from .helpers import * ('*' import everything; use of '*' is considered bad style)

In [ ]:
# (example:)
helpers.print_bold(f"This notebook/dashboard will visualize the contents from the database of profile {profile.name}")

# Subastk D2.a: Data acquisition

Task: check which output dict nodes returned by workflows, which had `StructureData` nodes as inputs are there in the database.

For example, for a successful `FleurScfWorkChain`, there are two return `Dict` nodes, one is linked with
`last_fleur_calc_output` and one with `output_scf_wc_para`.

If a `StructureData` is an input of such a workflow you can extract the formula, `uuid` and other information you need from the 
`StructureData` which is always linked into workflows via the link name `structure`.

All the user should have to say:
```python
source = generate_structure_property_pandas_source('<workflow_name>')
```


# Subtask D2.b: Interactive plot

Allow the user to choose, which properties to plot on what axis.

```python
xdata = source['distance']
ydata = source['energy']
```

Single bokeh scatter plot with histpgrams on both sides, hover tool should show 'input structure, formula, 
structure_uuid and dictnode uuid', as long as this information is available.

```python
bokeh_struc_prop_vis(xdata, ydata, src=source, **kwargs)
```